In [4]:
import Pkg.instantiate
instantiate()
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, istriu, qr
using CairoMakie

4×4 Matrix{Float64}:
 2.0  1.0  1.0  0.0
 4.0  3.0  3.0  1.0
 8.0  7.0  9.0  5.0
 6.0  7.0  9.0  8.0

4-element Vector{Float64}:
 2.0
 2.0
 2.0
 2.0

In [71]:
function classical_gram_schmidt(a)
    m = size(a, 1)
    n = size(a, 2)
    r = zeros(Float64, n, n)
    q = copy(a)
    for j=1:n
        for i=1:j-1, k=1:m
            r[i, j] += q[k, i] * q[k, j]
        end
        for i=1:j-1, k=1:m
            q[k,j] -= q[k,i] * r[i,j]
        end
        r[j,j] = norm( q[:, j] )
        q[:, j] /= r[j,j]
    end
    return q, r
end

# A = float([2 1 1 0; 4 3 3 1; 8 7 9 5; 6 7 9 8])
A = randn(30, 20) 
# display(A)
# display(A')

Q, R = classical_gram_schmidt(A)
# display(Q)
# display(Q' * Q)
# display(Q * R)
print(Q' * Q ≈ I)
print(Q * R ≈ A)

# print(norm(A-Q*R))
# display(A)
# display(A-(Q*R))


truetrue